In [1]:
import pandas as pd
import numpy as np
import cv2
import os
from tqdm import tqdm,trange
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face
import torch
from torchvision.transforms import ToTensor
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D, Bidirectional
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop, SGD
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import confusion_matrix
import time
from PIL import Image,ImageEnhance
import tensorflow as tf
from itertools import islice


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'



import warnings
warnings.filterwarnings("ignore")


Using TensorFlow backend.


In [2]:
max_frames = 10
max_df = 2
window_size= 5
supersteps = (max_frames-window_size+1)

df_train0 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_0/metadata.json')
df_train1 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_1/metadata.json')
df_train2 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_2/metadata.json')
df_train3 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_3/metadata.json')
df_train4 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_4/metadata.json')
df_train5 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_5/metadata.json')
df_train6 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_6/metadata.json')
df_train7 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_7/metadata.json')
df_train8 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_8/metadata.json')
df_train9 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_9/metadata.json')
df_train10 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_10/metadata.json')
df_train11 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_11/metadata.json')
df_train12 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_12/metadata.json')






df_train_all = [df_train0, df_train1, df_train2, df_train3, df_train4, 
                df_train5, df_train6, df_train7, df_train8, df_train9, 
                df_train10]

df_test_all = [df_train11, df_train12]


In [3]:
LABELS = ['REAL','FAKE']
device = 'cuda' if torch.cuda.is_available() else 'cpu'
resnet = InceptionResnetV1(pretrained='vggface2', device=device).eval()

In [4]:
"""function to read video frames from given paths"""
def read_img(path):
    frames = []
    for i in range(max_frames):
        frames.append(cv2.cvtColor(cv2.imread(path[i]),cv2.COLOR_BGR2RGB))
    return frames


In [5]:
def load_data(index, df_train):
    paths=[]
    y=[]

    df_train_values = list(df_train.columns.values)


    for value in df_train_values:
        image_paths=[]

        try:
            for num in range(max_frames):
                path = '/home/aelbakry1999/images/margin_0/dfdc_train_part_' + str(index) +"/"+ value.replace('.mp4', '') + '/frame' + str(num) +'.jpeg'
                image_paths.append(path)
                if not os.path.exists(path):
                    # print(path)
                    raise Exception
                
            paths.append(image_paths)
            y.append(LABELS.index(df_train[value]['label']))

        except Exception as err:
                # print(err)
            pass


    return paths, y


In [6]:
"""function to embed video frames with InceptionResnetV1"""
def embed(frames):
    faces_embedded = []
    tf_img = lambda i: ToTensor()(i).unsqueeze(0)
    embeddings = lambda input: resnet(input)

    with torch.no_grad():
        for faces in tqdm(frames):
            vid_embs = []
            for i in range(max_frames):
                t = tf_img(faces[i]).to(device)
                e = embeddings(t).squeeze().cpu().tolist()
                vid_embs.append(e)
            faces_embedded.append(vid_embs)

    return  faces_embedded


In [7]:
# all_paths=[]
# all_y=[]

# """Loading all paths and y_labels in df_train_all """
# # print("Loading training paths and y values from JSON files")
# for index in tqdm(range(np.shape(df_train_all)[0])):
#     path, labels = load_data(index, df_train_all[index])
#     all_paths.extend(path)
#     all_y.extend(labels)

# print(len(all_paths) )
# print(len(all_y) )


In [8]:
# paths = all_paths[:int(len(all_paths)*0.8)] 
# y = all_y[:int(len(all_y)*0.8)] 

# paths_test = all_paths[int(len(all_paths)*0.8):] 
# y_test = all_y[int(len(all_y)*0.8):] 


In [9]:
paths=[]
y=[]

"""Loading all paths and y_labels in df_train_all """
# print("Loading training paths and y values from JSON files")
for index in tqdm(range(np.shape(df_train_all)[0])):
    path, labels = load_data(index, df_train_all[index])
    paths.extend(path)
    y.extend(labels)

print(len(paths) )
print(len(y) )

100%|██████████| 11/11 [00:01<00:00,  9.69it/s]

16129
16129


In [10]:
paths_test=[]
y_test=[]

"""Loading all paths and y_labels in df_train_all """
# print("Loading training paths and y values from JSON files")
for index in tqdm(range(np.shape(df_test_all)[0])):
    path, labels = load_data(index, df_train_all[index])
    paths_test.extend(path)
    y_test.extend(labels)

print(len(paths_test) )
print(len(y_test) )


100%|██████████| 2/2 [00:00<00:00, 13.79it/s]

2055
2055


In [11]:
print('There are '+str(y.count(1))+' fake train samples')
print('There are '+str(y.count(0))+' real train samples')
print('There are '+str(y_test.count(1))+' fake train samples')
print('There are '+str(y_test.count(0))+' real train samples')

There are 14076 fake train samples
There are 2053 real train samples
There are 1921 fake train samples
There are 134 real train samples


In [12]:
"""Underbalancing training """

import random
real=[]
fake=[]
for m,n in zip(paths,y):
    if n==0:
        real.append(m)
    else:
        fake.append(m)
fake=random.sample(fake,len(real))
paths,y=[],[]
for x in real:
    paths.append(x)
    y.append(0)
for x in fake:
    paths.append(x)
    y.append(1)

In [13]:
"""Underbalancing validation/ test"""

import random
real=[]
fake=[]
for m,n in zip(paths_test,y_test):
    if n==0:
        real.append(m)
    else:
        fake.append(m)
fake=random.sample(fake,len(real))
paths_test,y_test=[],[]
for x in real:
    paths_test.append(x)
    y_test.append(0)
for x in fake:
    paths_test.append(x)
    y_test.append(1)

In [14]:
print('There are '+str(y.count(1))+' fake train samples')
print('There are '+str(y.count(0))+' real train samples')
print('There are '+str(y_test.count(1))+' fake train samples')
print('There are '+str(y_test.count(0))+' real train samples')

There are 2053 fake train samples
There are 2053 real train samples
There are 134 fake train samples
There are 134 real train samples


In [15]:
paths = np.array(paths)
y = np.array(y)


paths_test = np.array(paths_test)
y_test = np.array(y_test)

In [16]:
y = to_categorical(y, num_classes=2) #convert y training to one hot encodings
# y_test = to_categorical(y_test, num_classes=2) #convert y testing to one hot encodings

In [17]:
X=[]
# print("Loading frames training")
for img in tqdm(paths):
    X.append(read_img(img))
    
    
X_test=[]
# print("Loading frames testing")
for img in tqdm(paths_test):
    X_test.append(read_img(img))

100%|██████████| 268/268 [00:01<00:00, 162.69it/s]


In [18]:
train_size = len(X)
test_size = len(X_test)

In [19]:
print(y.shape)
print(y_test.shape)


(4106, 2)
(268,)


In [ ]:
X_embedded = embed(X)

 58%|█████▊    | 2402/4106 [08:40<05:43,  4.97it/s]

In [ ]:
X_test_embedded = embed(X_test)

In [ ]:
def window(seq, n):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [ ]:
X_rolled = []

for i in range(len(X_embedded)):
        rolled = window(X_embedded[i], n=window_size)
        X_rolled.append(list(rolled))
        
        
X_test_rolled = []

for i in range(len(X_test_embedded)):
        rolled = window(X_test_embedded[i], n=window_size)
        X_test_rolled.append(list(rolled))       

In [ ]:
print(np.shape(X_rolled))
print(y.shape)

print(np.shape(X_test_rolled))
print(y_test.shape)

In [ ]:
X_rolled = np.reshape(X_rolled, (train_size, supersteps , window_size , 512))
X_test_rolled = np.reshape(X_test_rolled, (test_size, supersteps , window_size , 512))


In [ ]:
def lstm():
    """Build a simple LSTM network. On the training sample"""
    # Model.
    model = Sequential()
    model.add(TimeDistributed(LSTM(2048, return_sequences=False), input_shape=( supersteps , window_size , 512 ) ))
    model.add(Dropout(0.5)) #large dropout
    model.add(Flatten())
    model.add((Dense(512, activation='relu')))
    model.add(Dropout(0.5)) #large dropout
    model.add(Dense(2, activation='softmax'))

    return model

model = lstm()

optimizer = Adam(lr=1e-5*10, decay=1e-6)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())


In [ ]:
history = model.fit(X_rolled, y, epochs=30, batch_size=supersteps*10, shuffle=True)
model.save_weights("model.h5")


In [ ]:
y_preds = model.predict_classes(X_test_rolled)


print(y_preds.shape)

In [ ]:
print(y_test.shape)
print(y_preds.shape)

In [ ]:
print("Predictions", y_preds)
print("True Labels", y_test)


In [ ]:
conf_matrix = confusion_matrix(y_test, y_preds)
conf_matrix

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_preds).ravel()


print("-------------- Confusion Matrix -------------- ")
print(conf_matrix)


print('True Positives: {}, True Negatives: {}, False Positives: {}, False Negatives: {}'.format(tp, tn, fp, fn))

precision = tp / (tp + fp)
accuracy = (tp + tn) / (tp + tn + fp + fn)
recall = tp / (tp + fn)
f1 = 2*(precision*recall)/ (precision+recall)
print("-------------- Model Scores -------------- ")
print('Precision: {}, Accuracy: {}, Recall: {}, F1-score: {}'.format(precision, accuracy, recall, f1))

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')


plt.subplot(2, 1, 2)
plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('/home/aelbakry1999/Results/accuracy_loss.png')